In [47]:
import pandas as pd
from PySAM.InvCecCg import new as InvCecCg
from itertools import product
from pvlib.inverter import fit_sandia

In [3]:
inverter_columns = ['Name', 'Vac', 'Pso', 'Paco', 'Pdco', 'Vdco', 'C0', 'C1', 'C2', 'C3',
       'Pnt', 'Vdcmax', 'Idcmax', 'Mppt_low', 'Mppt_high', 'CEC_Date',
       'CEC_Type', 'CEC_hybrid']

In [4]:
cec_inverters = pd.read_excel('/home/allen/Downloads/Grid_Support_Inverter_List_Full_Data_ADA.xlsm',
                              engine='openpyxl', skiprows=[*range(14), 15], usecols=range(60))

In [5]:
cec_inverters['Name'] = cec_inverters.apply(lambda row: f"{row['Manufacturer Name']} {row['Model Number1']}", axis=1)

In [56]:
inverter = cec_inverters.loc[cec_inverters['Model Number1'].str.contains('PV4400')].squeeze()

In [11]:
voltages = [k for k in cec_inverters.columns if k.startswith('Voltage ')]

powers = [k for k in cec_inverters.columns if k.startswith('Power Level')]

efficiencies = [k for k in cec_inverters.columns if k.startswith('Efficiency @V') and '6' not in k]

In [16]:
curve_keys = [[*v_p, e] for v_p, e in zip(product(voltages, powers), efficiencies)]
curve_keys = [[p, v, e] for v, p, e in curve_keys]

In [25]:
def get_curve_pvlib(inverter: pd.Series):
    curve = pd.DataFrame(columns=['Pac', 'Vdc', 'Eff'], data=[[inverter[k] for k in row] for row in curve_keys])

    curve['Eff'] *= 1e-2

    curve['Pdc'] = curve['Pac'] / curve['Eff']
    curve['Vlvl'] = curve['Vdc'].replace(
        curve['Vdc'].min(), 'Vmin').replace(
        curve['Vdc'].median(), 'Vnom').replace(
        curve['Vdc'].max(), 'Vmax')
    return curve
    
    

def get_params_pvlib(inverter: pd.Series):
    curve = get_curve(inverter)
    return fit_sandia(
        ac_power=curve['Pac'] * 1e3,
        dc_power=curve['Pdc'] * 1e3,
        dc_voltage=curve['Vdc'],
        dc_voltage_level=curve['Vlvl'],
        p_ac_0=inverter['Maximum Continuous Output Power at Unity Power Factor'] * 1e3,
        p_nt=inverter['Night Tare Loss'],
    )

In [50]:
def get_params_sam(inverter: pd.Series):
    curve = [[inverter[k] for k in row] for row in curve_keys]
    solver = InvCecCg()

    solver.Common.assign({
        'inv_cec_cg_paco': inverter['Maximum Continuous Output Power at Unity Power Factor'] * 1e3,
        'inv_cec_cg_sample_power_units': 1,
        'inv_cec_cg_test_samples': curve
    })

    solver.execute()
    return {
        k: v
        for k, v in solver.export()['Outputs'].items()
        if k in {'Pdco', 'Pso', 'Vdco', 'c0', 'c1', 'c2', 'c3'}
    }

In [102]:
def cec_row_to_sam_row(cec_row: pd.Series, get_params) -> pd.Series:
    params = get_params(cec_row)
    return pd.Series({
        'Name': f"{cec_row['Manufacturer Name']} {cec_row['Model Number1']}",
        'Vac': cec_row['Nominal Voltage'],
        'Vdco': cec_row['Voltage Nominal'],
        'Vdcmax': cec_row['Voltage Maximum'],
        'Pnt': cec_row['Night Tare Loss'],
        'Paco': cec_row['Maximum Continuous Output Power at Unity Power Factor'] * 1e3,
        'Pdco': params['Pdco'],
        'Pso': params['Pso'],
        'C0': params['c0'],
        'C1': params['c1'],
        'C2': params['c2'],
        'C3': params['c3'],
        'Idcmax': params['Pdco'] / params['Vdco'],
        'Mppt_low': cec_row['Voltage Minimum'],
        'Mppt_high': cec_row['Voltage Maximum'],
        'CEC_Date': inverter['Grid Support Listing Date'].date().isoformat(),  # TODO: or latest update
        'CEC_Type': 'Grid Support',
        'CEC_hybrid': 'N'
    })[inverter_columns]
    

In [103]:
sam_inverter = cec_row_to_sam_row(inverter, get_params_sam)

In [104]:
sam_inverter.to_frame().transpose().to_csv('production/nrel_sam/CEC_Inverters.csv', mode='a', header=False, index=False)